In [85]:
import collections
collections.Callable = collections.abc.Callable
import pandas as pd
import numpy as np
import requests
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup, NavigableString
import time
import csv
import os

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from selenium.webdriver.common.by import By
import urllib

In [86]:
#!pip install selenium          

In [87]:
#!pip install selenium webdriver-manager pandas

In [96]:
relative_path_plant = "../data_indiaplant_1_3000.csv"
relative_path_img_data = "./data_indiaplant_image_1_200.csv"

absolute_path_plant = os.path.join(os.getcwd(), relative_path_plant)
absolute_path_img_data = os.path.join(os.getcwd(), relative_path_img_data)

In [89]:
data = pd.read_csv(absolute_path_plant, index_col=0)
data

,Plant_Id,Scientific_Name,Common_Name,Regional_Name,Category,Family,Light,Water,Primary_Grown_for,Flowering_Season,Foliage_Color,Height_or_Length,Spread_or_Width,Plant_Form,Lifespan,Special_Feature,Plant_Description,Growing_Tips
0,1,Epiphyllum oxypetalum,"Queen Of The Night, Dutchmans-pipe Cactus",Marathi - Brahma Kamal,"Cacti & Succulents, Shrubs, Indoor Plants",Rhipsalidopsis,"Sun growing, Semi shade","Normal, Can tolerate less",Flowers,"May, June, July, August, September",Green,1 to 2 meters,1 to 2 meters,"Irregular, Upright or Erect",NaN,"Fragrant flowers or leaves, Auspicious or Feng...","Origin - Mexico to Brazil, Large, flat, thin, ...",Bright indirect sunlight to direct sunlight is...
1,2,Euphorbia grandicornis,Cow Horn Euphorbia,NaN,"Cacti & Succulents, Shrubs",Euphorbiaceae or Poinsettia family,"Sun growing, Semi shade","Normal, Can tolerate less",Foliage,"September, October, November, Flowers are inco...",Green,1 to 2 meters,50 cms to 100 cms,Upright or Erect,NaN,"Animals will not eat, Thorny or Spiny","Nation-Africa, Asia, Shrub grow-3 feet tall, B...","Full sun min 4 hours of sunlight, Sand in pott..."
2,4,Euphorbia milli thai hybrid,Crown Of Thorns New Hybrids,NaN,"Cacti & Succulents, Shrubs, Flowering Pot Plants",Euphorbiaceae or Poinsettia family,"Sun growing, Semi shade","Normal, Can tolerate less",Flowers,"Year-around flowering, Flowers in flushes thro...",Green,1 to 2 meters,50 cms to 100 cms,Upright or Erect,NaN,"Auspicious or Feng Shui plant, Good for Hedges...",This page is dedicated to no particular variet...,Full to diffused sunlight best. Minimum 3 to 4...
3,5,Euphorbia milli speciosa rubra,Crown Of Thorns - Red,NaN,"Cacti & Succulents, Flowering Pot Plants, Shrubs",Euphorbiaceae or Poinsettia family,"Sun growing, Semi shade","Normal, Can tolerate less",Flowers,"Year-around flowering, Flowers in flushes thro...",Green,50 cms to 100 cms,50 cms to 100 cms,Spreading,NaN,"Good for Hedges and Borders, Good for Edges ie...","A small flowered type of the euphorbia milli.,...",Full to diffused sunlight best. Minimum 3 to 4...
4,7,Euphorbia milli thai hybrid orange,Euphorbia Milli Thai Hyb Orange,NaN,"Cacti & Succulents, Shrubs, Flowering Pot Plants",Euphorbiaceae or Poinsettia family,"Sun growing, Semi shade","Normal, Can tolerate less",Flowers,"Year-around flowering, Flowers in flushes thro...",Green,1 to 2 meters,50 cms to 100 cms,Upright or Erect,NaN,"Auspicious or Feng Shui plant, Good for Hedges...",Extremely large flowered plants of the Thai go...,Full to diffused sunlight best. Minimum 3 to 4...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
905,2996,Garcinia indica variegated,Garcinia Variegated,"Hindi - Kokum Variegated, Marathi - Bheranda, ...","Trees, Medicinal Plants, Shrubs",Guttiferae or Nutmeg family,"Sun growing, Semi shade","Normal, Can tolerate more",Foliage,Flowers are inconspicuous,"Variegated, Green, White",4 to 6 meters,2 to 4 meters,Upright or Erect,NaN,"Indigenous (native to India), Auspicious or Fe...",This is a variegated leaf form of the regular ...,"Grows best in red lateratic fertile soils., Re..."
906,2997,Ficus cyatbistipula,NaN,NaN,"Shrubs, Trees",Moraceae or Fig family,"Sun growing, Semi shade","Normal, Can tolerate more",Foliage,Flowers are inconspicuous,Green,4 to 6 meters,2 to 4 meters,Upright or Erect,NaN,"Good for screening, Good for Hedges and Border...","A large leaved ficus., It can be distinguished...",As with most ficus this one too is easy to gro...
907,2998,Zephyranthes candida flava,Zephyranthes Cream Color,NaN,"Lilies & Bulbous plants, Groundcovers, Bamboos...",Amarylidaceae or Agave or Amaryllis family,"Sun growing, Semi shade","Normal, Can tolerate less, Can tolerate more",Flowers,"Year-around flowering, Flowers in flushes thro...",Green,Less than 50 cms,Less than 50 cms,"Spreading, Upright or Erect",NaN,"Good for Edges ie very small hedge or border, ...","Native of Americas., Off white flowers that ar...","Plants are very hardy and 

In [90]:
def initialize_driver():
    # Initialize the Chrome webdriver using ChromeDriverManager
    driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
    return driver

In [91]:
# Create a directory to store the scraped images
output_dir = './Scraped_Images'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [92]:
def image_scraping(df):
    # Initialize the Chrome webdriver using ChromeDriverManager
    driver = initialize_driver()

    # Create a directory to store the scraped images
    output_dir = './Scraped_Images'
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    paths = []
    # Iterate through each row in the dataframe
    for index, row in df.iterrows():
        plant_id = row['Plant_Id']
        scientific_name = row['Scientific_Name']

        # Create a directory for each plant
        plant_dir = os.path.join(output_dir, f'{plant_id}_{scientific_name}')
        if not os.path.exists(plant_dir):
            os.makedirs(plant_dir)
        query = scientific_name + " plant leaves and flowers"
        # Construct the search query URL with the scientific name
        query_url = ("https://www.google.com/search?q={s}&tbm=isch&tbs=sur%3Afc&hl=en&ved=0CAIQpwVqFwoTCKCa1c6s4-oCFQAAAAAdAAAAABAC&biw=1251&bih=568")
        driver.get(query_url.format(s=query))

        driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
        time.sleep(5)

        # Find all <g-img> tags with class "mNsIhb"
        g_img_results = driver.find_elements(By.CLASS_NAME, "mNsIhb")
        img_paths = []

        try:
            # Extract image sources and download the images
            counter = 0  # Initialize counter for unique filenames
            for g_img in g_img_results:
                img_results = g_img.find_elements(By.CLASS_NAME, "YQ4gaf")
                for img in img_results:
                    src = img.get_attribute('src')
                    if src and counter < 5:  # Download only the first 5 images
                        img_path = os.path.join(plant_dir, f'{scientific_name}_{counter}.jpg')
                        urllib.request.urlretrieve(src, img_path)
                        img_paths.append(img_path)
                        counter += 1  # Increment counter for next image
            paths.append(img_paths)
        except Exception as e:
            paths.append([])
            print(f"Error occurred for plant {plant_id} - {scientific_name}: {e}")

    # Quit the driver after extraction
    driver.quit()    
    return paths

In [93]:
data_new = data[0:200].copy()

In [94]:
len(data_new)

200

In [95]:
# Add the image paths to the dataframe
data_new_path = image_scraping(data_new)

KeyboardInterrupt: 

In [ ]:
len(data_new_path)

In [79]:
data_new['Image_Paths'] = data_new_path
data_new

,Plant_Id,Scientific_Name,Common_Name,Regional_Name,Category,Family,Light,Water,Primary_Grown_for,Flowering_Season,Foliage_Color,Height_or_Length,Spread_or_Width,Plant_Form,Lifespan,Special_Feature,Plant_Description,Growing_Tips,Image_Paths
0,1,Epiphyllum oxypetalum,"Queen Of The Night, Dutchmans-pipe Cactus",Marathi - Brahma Kamal,"Cacti & Succulents, Shrubs, Indoor Plants",Rhipsalidopsis,"Sun growing, Semi shade","Normal, Can tolerate less",Flowers,"May, June, July, August, September",Green,1 to 2 meters,1 to 2 meters,"Irregular, Upright or Erect",NaN,"Fragrant flowers or leaves, Auspicious or Feng...","Origin - Mexico to Brazil, Large, flat, thin, ...",Bright indirect sunlight to direct sunlight is...,[./Scraped_Images/1_Epiphyllum oxypetalum/Epip...
1,2,Euphorbia grandicornis,Cow Horn Euphorbia,NaN,"Cacti & Succulents, Shrubs",Euphorbiaceae or Poinsettia family,"Sun growing, Semi shade","Normal, Can tolerate less",Foliage,"September, October, November, Flowers are inco...",Green,1 to 2 meters,50 cms to 100 cms,Upright or Erect,NaN,"Animals will not eat, Thorny or Spiny","Nation-Africa, Asia, Shrub grow-3 feet tall, B...","Full sun min 4 hours of sunlight, Sand in pott...",[./Scraped_Images/2_Euphorbia grandicornis/Eup...


In [ ]:
data_new.to_csv(absolute_path_img_data, index=False)

In [53]:
# driver = initialize_driver()
# url = ("https://www.google.com/search?q={s}&tbm=isch&tbs=sur%3Afc&hl=en&ved=0CAIQpwVqFwoTCKCa1c6s4-oCFQAAAAAdAAAAABAC&biw=1251&bih=568")
# driver.get(url.format(s='Hibiscus plant leaves and flowers'))
# driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
# time.sleep(5)
# # Find all <g-img> tags with class "mNsIhb"
# g_img_results = driver.find_elements(By.CLASS_NAME, "mNsIhb")
# src = []

# for g_img in g_img_results:
#     # Find <img> tags with class "YQ4gaf" inside each <g-img> tag
#     img_results = g_img.find_elements(By.CLASS_NAME, "YQ4gaf")
#     for img in img_results:
#         src.append(img.get_attribute('src'))

# for i in range(min(10, len(src))):    
#     urllib.request.urlretrieve(str(src[i]),"./sample_image/rose{}.jpg".format(i))
    
# # Quit the driver after extraction
# driver.quit()